## 动态规划
1. 重复做的事缓存进表，不再展开
2. 从尝试入手

### 1.机器人问题
* 一共有N个位置，有个机器人的位置在start，目标位置在aim，需要走的步数是K。
* 小人如果步数剩余必须动，在1位置下一步必须走到2，在N位置下一步必须走到N-1。其余位置可左可右。
* 返回start到aim的所有走法

In [32]:
#尝试过程
#机器人当前位置cur，还有rest步要走，有1~N的位置。
#返回从cur出发，走rest步后，最终停到aim的方法数？
def process1(cur, rest, aim, N):
    #走完了
    if rest == 0:
        return 1 if cur == aim else 0
    #rest > 0
    if cur == 1:
        return process1(2, rest-1, aim, N)
    if cur == N:
        return process1(N-1, rest-1, aim, N)
    #中间位置上
    return process1(cur-1, rest-1, aim, N) + process1(cur+1, rest-1, aim, N)

def ways1(N, start, aim, K):
    if N < 2 or start < 1 or start > N or aim < 1 or aim > N or K < 1:
        return -1
    return process1(start, K, aim, N)

In [18]:
#尝试改傻缓存 == 记忆化搜索 == 从顶向下的动态规划

In [31]:
def process2(cur, rest, aim, N, dp):
    if dp[cur][rest] != -1:
        return dp[cur][rest]
    #走完了
    if rest == 0:
        ans = 1 if cur == aim else 0
    #rest > 0
    elif cur == 1:
        ans = process2(2, rest-1, aim, N, dp)
    elif cur == N:
        ans = process2(N-1, rest-1, aim, N, dp)
    #中间位置上
    else:
        ans = process2(cur-1, rest-1, aim, N, dp) + process2(cur+1, rest-1, aim, N, dp)
    #注意修改缓存表
    dp[cur][rest] = ans
    return ans

def ways2(N, start, aim, K):
    if N < 2 or start < 1 or start > N or aim < 1 or aim > N or K < 1:
        return -1
    dp = [[-1] * (K+1) for _ in range(aim+2)]
    #dp[i][j]==-1 -> process1没算过
    return process2(start, K, aim, N, dp)

In [30]:
# 根据位置依赖直接填表，避免递归
def ways3(N, start, aim, K):
    if N < 2 or start < 1 or start > N or aim < 1 or aim > N or K < 1:
        return -1
    dp = [[0] * (K+1) for _ in range(aim+2)]
    #dp[i][j]==-1 -> process1没算过
    dp[aim][0] = 1 #dp[...][0] = 0, dp[aim][0]=1
    for rest in range(1, K+1):
        dp[1][rest] = dp[2][res-1]
        for cur in range(2, N):
            dp[cur][res] = dp[cur-1][res-1]+dp[cur+1][res-1]
        dp[N][rest-1] = dp[N-1][rest-1]
        
    return dp[start][K]

In [29]:
print(ways1(5,2,4,6))
print(ways2(5,2,4,6))
print(ways2(5,2,4,6))

13
13
13


### 2.纸牌问题
* 给定一个整数arr(>=0)，代表不同数值的纸牌排成一行。玩家AB依次拿走左侧或右侧的牌
* 规定A先拿，B后拿。AB都很聪明，都拿最优解
* 返回谁会赢

In [34]:
#根据规则返回获胜者的分数
def win1(arr):
    if not arr:
        return 0
    first = f(arr,0,len(arr)-1)
    second = g(arr,0,len(arr)-1)
    return max(first, second)

#先手
def f(arr, L, R):
    if L==R:
        return arr[L]
    p1 = arr[L] + g(arr, L+1, R) #拿走L位置
    p2 = arr[R] + g(arr, L, R-1) #拿走R位置
    return max(p1,p2)

#后手
def g(arr, L, R):
    if L==R:
        return 0
    p1 = f(arr, L+1, R) #对手做完决定，拿到最好
    p2 = f(arr, L, R-1)
    return min(p1,p2) #对手做决定，只能拿小的那个。两种可能里的最小能拿到最好的那个！
        

In [43]:
#傻缓存法,f/g分别建表
import numpy as np
def win2(arr):
    if not arr:
        return 0
    N = len(arr)
    fmap = np.ones((N,N)) * (-1)
    gmap = np.ones((N,N)) * (-1)
    first = f2(arr,0,len(arr)-1, fmap, gmap)
    second = g2(arr,0,len(arr)-1, fmap, gmap)
    return max(first, second)

#先手
def f2(arr, L, R, fmap, gmap):
    if fmap[L][R] != -1:
        return fmap[L][R]
    if L==R:
        ans = arr[L]
    else:
        p1 = arr[L] + g2(arr, L+1, R, fmap, gmap) #拿走L位置
        p2 = arr[R] + g2(arr, L, R-1, fmap, gmap) #拿走R位置
        ans = max(p1,p2)
    fmap[L][R] = ans
    return ans

#后手
def g2(arr, L, R, fmap, gmap):
    if gmap[L][R] != -1:
        return gmap[L][R]
    if L==R:
        ans = 0
    else:
        p1 = f2(arr, L+1, R, fmap, gmap) #对手做完决定，拿到最好
        p2 = f2(arr, L, R-1, fmap, gmap)
        ans = min(p1, p2)
    gmap[L][R] = ans
    return ans #对手做决定，只能拿小的那个。两种可能里的最小能拿到最好的那个！
        

In [47]:
#看base case填表
def win3(arr):
    if not arr:
        return 0
    N = len(arr)
    fmap = np.ones((N,N)) * (-1)
    gmap = np.ones((N,N)) * (-1)
    for i in range(N):
        fmap[i][i] = arr[i]
        gmap[i][i] = 0    
    for startCol in range(1, N):
        L = 0
        R = startCol
        while R < N:
            fmap[L][R] = max(arr[L] + gmap[L+1][R], arr[R] + gmap[L][R-1])
            gmap[L][R] = min(fmap[L+1][R], fmap[L][R-1])
            L += 1
            R += 1
    return max(fmap[0][N-1], gmap[0][N-1])

In [48]:
arr =[5,7,4,5,8,1,6,0,3,4,6,1,7]
print(win1(arr))
print(win2(arr))
print(win3(arr))

32
32.0
32.0


In [42]:
np.ones((2,2)) * -1

array([[-1., -1.],
       [-1., -1.]])